### Import lib

In [ ]:
# With OS
from os import sep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd


In [ ]:
# With colab
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException, UnexpectedAlertPresentException
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd


### Setup and open chrome


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


### Get transcript student

In [ ]:
# get input field and submit
def getSV(msv) :
    msv_field = driver.find_element_by_id('ctl00_ContentPlaceHolder1_ctl00_txtMaSV')
    msv_field.send_keys(msv)
    
    search_btn = driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_btnOK"]')
    search_btn.click()
   
    # view all result
    # view_all = driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_lnkChangeview2"]')
    # view_all.click()

# get transcript
def getResult(msv) :
    page_source = BeautifulSoup(driver.page_source, "lxml")
    table_src = page_source.find_all("table", class_="view-table")[0]

    table_rows = table_src.find_all("tr")
    del table_rows[0]

    for tr in table_rows:
        district_demo = tr.find_all("td")
        row_demo = [tr.text for tr in district_demo]
        row_demo.append(msv)
        listTranscript.append(row_demo) 

# get profile
def getInfo() :
    page_source = BeautifulSoup(driver.page_source, "lxml")
    table_src = page_source.find_all("table")[2]

    table_rows = table_src.find_all("tr")

    for tr in table_rows:
        district_demo = tr.find_all("td")
        row_demo = [tr.text for tr in district_demo]
        listProfile.append(row_demo) 
    

def getSemesterResult(seme = '20201') :
    semester_field = driver.find_element_by_id('ctl00_ContentPlaceHolder1_ctl00_txtChonHK')
    semester_field.send_keys(seme)

    semester_btn = driver.find_element_by_id('ctl00_ContentPlaceHolder1_ctl00_btnChonHK')
    semester_btn.click()

    all_info = {}
    page_source = BeautifulSoup(driver.page_source)

    # personal info
    code = driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_ucThongTinSV_lblMaSinhVien"]').text
    all_info['code'] = code

    # point
    list_row = page_source.find_all('tr', class_='row-diem')
    for j, row in enumerate(list_row) :
        if j == len(list_row)/2 : break
        for i, cell in enumerate(row) :
            if i==2 : 
                subj = cell.text
            if i==11 : 
                point = cell.text
        while subj in all_info :
            subj = subj + '*'
        all_info[subj] = point  

    list_final = page_source.find_all('tr', class_='row-diemTK')
    for row in list_final :
        for n, val in enumerate(row.find('td')) :
            if n : point = val.text
            else : subj = val.text
        all_info[subj] = point  

    return all_info


###Setup input

In [ ]:
# input file
inputFile = pd.read_excel('/content/student.xlsx')
data =inputFile['code'].tolist()
data.sort()
# del data[:20013]

headings = ['STT', 'Subject', 'Subject Name', 'TC', '% KT', '% Thi', 'Điểm Chuyên Cần', 'Điểm Quá Trình', 'Thi L1', 'TK1(10)', 'Point', 'TK1(CH)', 'TK(CH)','Code']
listTranscript = []

headProfile = ["Name", "Value"]
listProfile =[]
for msv in data:
    url = "http://daotao.vnua.edu.vn/default.aspx?page=nhapmasv&flag=XemDiemThi"
    driver.get(url)
    # sleep(1)
    getSV(msv)
    # sleep(1)
    # pass alert box
    try :
        getInfo()
        # getResult(msv)
    except Exception as e:
        if  "Không tìm thấy mã vừa nhập" not in str(e) :
            print(msv,e)
        continue
# lean NaN value
outputTranscript = pd.DataFrame(data=listTranscript,columns=headings)
outputTranscript = outputTranscript.fillna('')

outputTranscript.to_csv('transcript.csv', index=False)

outputProfile = pd.DataFrame(data=listProfile,columns=headProfile)
outputTranscript = outputTranscript.fillna('')

outputProfile.to_csv('profile.csv', index=False)


In [ ]:
outputTranscript = pd.DataFrame(data=listTranscript,columns=headings)
outputTranscript = outputTranscript.fillna('')

outputTranscript.to_csv('transcript.csv', index=False)
outputTranscript

outputProfile = pd.DataFrame(data=listProfile,columns=headProfile)
outputProfile = outputProfile.fillna('')

outputProfile.to_csv('profile.csv', index=False)
outputProfile

In [ ]:
data = pd.read_excel('/content/student.xlsx')
ds =data['code'].tolist()

ds.sort()
del ds[:20013]
print(ds)
